# Análise Detalhada da Relação Taxa Amostral x Janela x F1-score

A ideia é fazer uma análise do <u>impacto dos parâmetros de Taxa Amostral e Janela</u> na qualidade dos modelos binários. 

Os dados utilizados neste estudo foram os obtidos no `estudo 17`.

Para isto, vamos dividir a análise em dois indicadores:

1. Melhor Taxa Amostral global e a relação de melhores Janelas por Aparelho;
2. Melhores combinações de Taxa Amostral e Janelas por Aparelho.

# Configurações

In [1]:
import os
import sys
import gc
from pprint import pprint
from collections import Counter
import copy
import warnings
warnings.filterwarnings(action="ignore")

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from IPython.display import display
%load_ext watermark

# TODO: implementar rotina na classe PyNILM.utils
def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

def listar_variaveis_memoria(ambiente):
    print("* Variáveis instanciadas em memória:")
    print("---")
    total = 0
    for name, size in sorted(((name, sys.getsizeof(value)) for name, value in ambiente.items()),
                             key= lambda x: -x[1])[:10]:
        total += size
        print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))
    print("---")
    print("Total:", sizeof_fmt(total))
    
# TODO: implementar na classe utils
def highlight_col(x):
    r = 'background-color: #D9D9D9'
    df1 = pd.DataFrame('', index=x.index, columns=x.columns)
    df1.iloc[:, -2] = r
    return df1   

In [2]:
# CONSTANTES FUNDAMENTAIS DE ORGANIZACAO DE PASTAS/ARQUIVOS

# Path do arquivo H5 (base REDD ja preparada p/ NILMTK) e outros insumos fundamentais
caminho_dados = "D:/Projetos/phd-thesis/datasets/"

# Definir diretorios onde iremos salvar os insumos gerados do notebook (dados, imagens, etc.)
caminho_dados_notebook = os.path.join(caminho_dados, "17") # Num. notebook
if not os.path.isdir(caminho_dados_notebook):
    os.makedirs(caminho_dados_notebook)
caminho_imagens_notebook = os.path.join(caminho_dados_notebook, "imagens") # Num. notebook
if not os.path.isdir(caminho_imagens_notebook):
    os.makedirs(caminho_imagens_notebook)

# Path do arquivo H5 (base REDD ja preparada p/ NILMTK)
caminho_redd = os.path.join(caminho_dados, "REDD/low_freq")

# Path completo do arquivo REDD
arquivo_dataset = os.path.join(caminho_redd, "redd.h5")

# VARIAVEL AUXILIAR
# Path dos arquivos relacionados as janelas
caminho_janelas = os.path.join(caminho_redd, "../../phd")
if not os.path.isdir(caminho_janelas):
    os.makedirs(caminho_janelas)

# Dados

In [3]:
# Checar se df dos resultados foi carregado
if not 'df_resultados' in locals():
    df_resultados = pd.read_csv(os.path.join(caminho_dados_notebook, "resultados_taxa-amostragem_janelas.csv"), index_col=[0])
df_resultados

,carga,taxa_amostragem,janela,loss,acuracia,precisao,recall,f1,f1_macro
0,sockets - 3,3,30,3.925722e-02,0.988928,0.00,0.00,0.00,49.72
1,sockets - 4,3,30,1.049300e-09,1.000000,0.00,0.00,0.00,100.00
2,light - 5,3,30,7.928869e-02,0.996503,0.00,0.00,0.00,49.91
3,ce_appliance - 6,3,30,1.013742e-09,1.000000,100.00,100.00,100.00,100.00
4,fridge - 7,3,30,4.661609e-01,0.762821,58.75,76.47,66.45,74.05
5,waste_disposal_unit - 8,3,30,3.368800e-02,0.998252,0.00,0.00,0.00,49.96
6,dish_washer - 9,3,30,9.707676e-02,0.984848,16.67,4.55,7.14,53.19
7,electric_furnace - 10,3,30,1.721915e-01,0.949883,37.50,3.57,6.52,51.97
8,light - 11,3,30,3.829945e-01,0.838578,64.40,61.88,63.12,76.39
9,sockets - 12,3,30,3.837796e-01,0.833916,55.88,6.57,11.76,51.30


# Análise

## Melhor Taxa Amostral global e relação de melhores Janelas por Aparelho

In [4]:
# Verificar o desempenho GERAL (todos as janelas/aparelhos) por taxa amostral
# (premissa basica inicial pro sistema de NILM)
df_resultados.groupby("taxa_amostragem").agg({
    "acuracia":["count","mean", "std","min","max"],
    "f1_macro":["count","mean", "std","min","max"]
})

acuracia                                    f1_macro  \
                   count      mean       std       min  max    count   
taxa_amostragem                                                        
3                    162  0.923950  0.088740  0.666667  1.0      162   
4                    162  0.916421  0.093649  0.527778  1.0      162   
5                    162  0.917006  0.093796  0.581395  1.0      162   
8                    162  0.911121  0.097661  0.500000  1.0      162   

                                                     
                      mean        std    min    max  
taxa_amostragem                                      
3                67.969506  18.785611  45.86  100.0  
4                67.641481  19.520736  39.23  100.0  
5                67.870062  19.413483  41.36  100.0  
8                70.021049  20.351877  35.71  100.0

**NOTA:** A melhor Taxa Amostral global:

* Considerando `Acurácia`: 5 segundos (utilizado no estudo anterior)
* Considerando `F1-score`: 8 segundos (adotado a partir deste estudo)

In [113]:
# Melhor taxa amostragem encontrada
melhor_taxa = 8

In [114]:
# Verificar as melhores janelas obtidas para a taxa amostral + benchmark para pesquisa
df_melhores_janelas = pd.DataFrame(data={},columns=df_resultados.columns)
for carga in df_resultados["carga"].unique():
    df = df_resultados.loc[
        (df_resultados["taxa_amostragem"]==melhor_taxa) & \
            (df_resultados["carga"]==carga),
        :
    ]
    df.reset_index(inplace=True, drop=True)

    df.sort_values(METRICA_COMPARATIVA["metrica"], ascending=False, inplace=True)
    melhor_score = df.iloc[0]
    df_melhores_janelas = df_melhores_janelas.append(melhor_score, ignore_index=True)

# Melhores janelas por aparelho (taxa fixa = 8)
df_melhores_janelas

,carga,taxa_amostragem,janela,loss,acuracia,precisao,recall,f1,f1_macro
0,sockets - 3,8,180,1.037,0.991,0.000,0.000,0.000,49.770
1,sockets - 4,8,30,0.000,1.000,0.000,0.000,0.000,100.000
2,light - 5,8,180,0.002,1.000,0.000,0.000,0.000,100.000
3,ce_appliance - 6,8,30,0.000,1.000,100.000,100.000,100.000,100.000
4,fridge - 7,8,360,0.000,1.000,100.000,100.000,100.000,100.000
5,waste_disposal_unit - 8,8,30,0.062,0.994,0.000,0.000,0.000,49.840
6,dish_washer - 9,8,900,0.143,0.909,33.330,100.000,50.000,72.500
7,electric_furnace - 10,8,60,0.349,0.950,66.670,54.550,60.000,78.680
8,light - 11,8,900,0.204,0.909,85.710,85.710,85.710,89.520
9,sockets - 12,8,60,0.798,0.835,54.550,21.820,31.170,60.910


In [115]:
from itertools import repeat
comparativo = {"taxa": []}
comparativo

{'taxa': []}

In [116]:
# Resumo por metrica
comparativo["taxa"].append("fixa")
for metrica in ["acuracia","precisao","recall","f1","f1_macro","loss"]:
    print("*", metrica+":")
    print("---")
    print("mean: {:.3f}".format(df_melhores_janelas[metrica].mean()) )
    print("std : {:.3f}".format(df_melhores_janelas[metrica].std()) )
    print("max : {:.3f}".format(df_melhores_janelas[metrica].max()) )
    print("min : {:.3f}".format(df_melhores_janelas[metrica].min()) )    
    
    if not metrica in comparativo:
        comparativo[metrica] = []
        
    comparativo[metrica].append(df_melhores_janelas[metrica].mean())
# pd.set_option('display.float_format', lambda x: '%.3f' % x)
# df_melhores_janelas.groupby(["taxa_amostragem"]).agg({
#     "acuracia": ["mean","std","min","max"],
#     "precisao": ["mean","std","min","max"],
#     "recall": ["mean","std","min","max"],
#     #"f1": ["mean","std","min","max"],
#     "f1_macro": ["mean","std","min","max"],
#     "loss": ["mean","std","min","max"],
# }).T

* acuracia:
---
mean: 0.955
std : 0.054
max : 1.000
min : 0.835
* precisao:
---
mean: 60.416
std : 42.651
max : 100.000
min : 0.000
* recall:
---
mean: 52.669
std : 39.638
max : 100.000
min : 0.000
* f1:
---
mean: 53.991
std : 38.820
max : 100.000
min : 0.000
* f1_macro:
---
mean: 83.784
std : 16.594
max : 100.000
min : 49.770
* loss:
---
mean: 0.757
std : 1.564
max : 6.734
min : 0.000


## Melhores combinações de Taxa Amostral e Janelas por Aparelho

In [117]:
# Verificar as melhores janelas obtidas para a taxa amostral + benchmark para pesquisa
df_melhores_taxas_janelas = pd.DataFrame(data={},columns=df_resultados.columns)
for carga in df_resultados["carga"].unique():
    df = df_resultados.loc[
        (df_resultados["carga"]==carga),
        :
    ]
    df.reset_index(inplace=True, drop=True)

    df.sort_values(METRICA_COMPARATIVA["metrica"], ascending=False, inplace=True)
    melhor_score = df.iloc[0]
    df_melhores_taxas_janelas = df_melhores_taxas_janelas.append(melhor_score, ignore_index=True)

# Melhores taxas e janelas por aparelho (taxa variavel)
df_melhores_taxas_janelas

,carga,taxa_amostragem,janela,loss,acuracia,precisao,recall,f1,f1_macro
0,sockets - 3,3,90,0.231,0.986,40.000,66.670,50.000,74.650
1,sockets - 4,3,30,0.000,1.000,0.000,0.000,0.000,100.000
2,light - 5,4,360,0.008,1.000,0.000,0.000,0.000,100.000
3,ce_appliance - 6,3,30,0.000,1.000,100.000,100.000,100.000,100.000
4,fridge - 7,8,1080,0.000,1.000,100.000,100.000,100.000,100.000
5,waste_disposal_unit - 8,4,900,2.371,0.791,11.110,50.000,18.180,53.090
6,dish_washer - 9,5,360,0.092,0.977,50.000,50.000,50.000,74.400
7,electric_furnace - 10,5,720,1.154,0.907,83.330,62.500,71.430,82.940
8,light - 11,8,900,0.204,0.909,85.710,85.710,85.710,89.520
9,sockets - 12,5,540,2.086,0.860,75.000,30.000,42.860,67.430


In [118]:
# Resumo por metrica
comparativo["taxa"].append("variavel")
for metrica in ["acuracia","precisao","recall","f1","f1_macro","loss"]:
    print("*", metrica+":")
    print("---")
    print("mean: {:.3f}".format(df_melhores_taxas_janelas[metrica].mean()) )
    print("std : {:.3f}".format(df_melhores_taxas_janelas[metrica].std()) )
    print("max : {:.3f}".format(df_melhores_taxas_janelas[metrica].max()) )
    print("min : {:.3f}".format(df_melhores_taxas_janelas[metrica].min()) )    
    
    comparativo[metrica].append(df_melhores_taxas_janelas[metrica].mean())
    
    print()

* acuracia:
---
mean: 0.951
std : 0.062
max : 1.000
min : 0.791

* precisao:
---
mean: 60.908
std : 41.183
max : 100.000
min : 0.000

* recall:
---
mean: 56.523
std : 37.492
max : 100.000
min : 0.000

* f1:
---
mean: 56.111
std : 37.781
max : 100.000
min : 0.000

* f1_macro:
---
mean: 87.533
std : 13.601
max : 100.000
min : 53.090

* loss:
---
mean: 0.730
std : 0.975
max : 2.895
min : 0.000



## Comparação: Taxa Fixa vs. Taxa Variável por Aparelho

In [125]:
df_comparativo = pd.DataFrame(comparativo)
df_comparativo

,taxa,acuracia,precisao,recall,f1,f1_macro,loss
0,fixa,0.955,60.416,52.669,53.991,83.784,0.757
1,variavel,0.951,60.908,56.523,56.111,87.533,0.730


In [126]:
df_comparativo.set_index("taxa").diff()

,acuracia,precisao,recall,f1,f1_macro,loss
taxa,,,,,,
fixa,nan,nan,nan,nan,nan,nan
variavel,-0.005,0.492,3.854,2.119,3.748,-0.027


## Comparação: Diferenças Considerando F1-score (macro)

In [146]:
df_comparacao2 = pd.DataFrame(
    np.hstack([
        df_melhores_janelas[["taxa_amostragem","janela",METRICA_COMPARATIVA["metrica"]]].values,
        df_melhores_taxas_janelas[["taxa_amostragem","janela",METRICA_COMPARATIVA["metrica"]]].values,
        np.expand_dims(
            df_melhores_taxas_janelas[METRICA_COMPARATIVA["metrica"]].values - \
                df_melhores_janelas[METRICA_COMPARATIVA["metrica"]].values, 
            axis=1
        ),
    ]),  
    index=df_melhores_taxas_janelas["carga"].values, 
    columns=[
        ["Taxa Fixa", "Taxa Fixa", "Taxa Fixa",
         "Taxa Variável", "Taxa Variável", "Taxa Variável", 
         "Diferença"],
        ["Taxa","Janela", METRICA_COMPARATIVA["rotulo"], 
         "Taxa","Janela", METRICA_COMPARATIVA["rotulo"], 
         METRICA_COMPARATIVA["rotulo"]]
    ]
)
df_comparacao2

Taxa Fixa                 Taxa Variável         \
                             Taxa Janela F1-macro          Taxa Janela   
sockets - 3                     8    180   49.770             3     90   
sockets - 4                     8     30  100.000             3     30   
light - 5                       8    180  100.000             4    360   
ce_appliance - 6                8     30  100.000             3     30   
fridge - 7                      8    360  100.000             8   1080   
waste_disposal_unit - 8         8     30   49.840             4    900   
dish_washer - 9                 8    900   72.500             5    360   
electric_furnace - 10           8     60   78.680             5    720   
light - 11                      8    900   89.520             8    900   
sockets - 12                    8     60   60.910             5    540   
washer_dryer - 13               8     30   84.510             3    900   
washer_dryer - 14               8    360   82.350             3    900   
light - 15                      8     60  100.000             5     30   
microwave - 16                  8    180   77.850             8    180   
light - 17                      8    720   85.160             5    900   
smoke_alarm - 18                8     30  100.000             8   1080   
light - 19                      8    720   88.640             5   1080   
unknown - 20                    8   1080   88.390             8   1080   

                                 Diferença  
                        F1-macro  F1-macro  
sockets - 3               74.650    24.880  
sockets - 4              100.000     0.000  
light - 5                100.000     0.000  
ce_appliance - 6         100.000     0.000  
fridge - 7               100.000     0.000  
waste_disposal_unit - 8   53.090     3.250  
dish_washer - 9           74.400     1.900  
electric_furnace - 10     82.940     4.260  
light - 11                89.520     0.000  
sockets - 12              67.430     6.520  
washer_dryer - 13        100.000    15.490  
washer_dryer - 14         89.540     7.190  
light - 15               100.000     0.000  
microwave - 16            77.850     0.000  
light - 17                88.330     3.170  
smoke_alarm - 18         100.000     0.000  
light - 19                89.450     0.810  
unknown - 20              88.390     0.000

In [153]:
# Apenas aparelhos do paper IEEE PES GM
df_comparacao2.loc[ df_comparacao2.index.isin([
        "fridge - 7", "dish_washer - 9", "washer_dryer - 13", 
        "washer_dryer - 14", "microwave - 16"
    ]),:]

Taxa Fixa                 Taxa Variável                  \
                       Taxa Janela F1-macro          Taxa Janela F1-macro   
fridge - 7                8    360  100.000             8   1080  100.000   
dish_washer - 9           8    900   72.500             5    360   74.400   
washer_dryer - 13         8     30   84.510             3    900  100.000   
washer_dryer - 14         8    360   82.350             3    900   89.540   
microwave - 16            8    180   77.850             8    180   77.850   

                  Diferença  
                   F1-macro  
fridge - 7            0.000  
dish_washer - 9       1.900  
washer_dryer - 13    15.490  
washer_dryer - 14     7.190  
microwave - 16        0.000

# Conclusões

1. A escolha de taxa amostral variável junto as janelas também variáveis por aparelho demonstrou ser útil para a melhoria dos classificadores, resultando em um `f1-macro ~3.75%` superior a abordagem considerando uma taxa fixa (hipótese inicial);

2. Quando comparado o resultado desta metodologia apenas nas cargas de maior consumo da residência 3 (consideradas no paper *IEEE PES GM 2020), observa-se ganhos relevantes, de até ~15.4% (wash dryer 1) em relação a hipótese de taxa fixa.

Sendo assim, observa-se a eficácia em adotar adotar uma metodologia híbrida para escolha de taxa amostral e janelas por aparelho, a qual deverá ser considerada para a continuidade da pesquisa.

# Fim.

In [155]:
%watermark -a "Diego Luiz Cavalca" -u -n -t -z -v -m -g

Diego Luiz Cavalca 
last updated: Thu Jul 09 2020 17:27:02 Hora oficial do Brasil 

CPython 3.6.7
IPython 7.6.1

compiler   : MSC v.1900 64 bit (AMD64)
system     : Windows
release    : 10
machine    : AMD64
processor  : Intel64 Family 6 Model 158 Stepping 9, GenuineIntel
CPU cores  : 8
interpreter: 64bit
Git hash   : 3c1da0276e8460bc9159fe2e8d6f1a4c23b316ba
